In [160]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from matplotlib import pyplot
import operator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import roc_auc_score

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import re
import sys
import pickle
from tqdm import tqdm
import os

In [161]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [162]:
df1 = pd.read_csv('/content/drive/My Drive/train_cs.csv')
print(df1.columns)

Index(['UserID', 'YOB', 'Gender', 'Income', 'HouseholdStatus',
       'EducationLevel', 'Party', 'Happy', 'Q124742', 'Q124122',
       ...
       'Q99716', 'Q99581', 'Q99480', 'Q98869', 'Q98578', 'Q98059', 'Q98078',
       'Q98197', 'Q96024', 'votes'],
      dtype='object', length=110)


**As per the instructions in the class room instead of taking single observation, Here I am passing entore dataset**

In [163]:
def happiness_prediction(df1):

  #data preprocessing

  #dropping UserID as it is an unimportant feature
  df1.drop(['UserID'], axis = 1, inplace = True)
  #features with missing values between 5-20% missing (mean/median is to be used to fill NAN values)
  features_mean = []
  for i in (df1.columns):
    if(df1[i].isnull().sum()*100/len(df1)) >=5 and (df1[i].isnull().sum()*100/len(df1))<=20:
      features_mean.append(i)
  
  df1['YOB'] = 2014 - df1['YOB']

  df1 = df1.replace({'HouseholdStatus': {'Married (w/kids)' :3, 'Single (no kids)':0, 'Married (no kids)':2,'Domestic Partners (no kids)':4,'Domestic Partners (w/kids)':5, 'Single (w/kids)':1}})
  df1 = df1.replace({'EducationLevel': {'Master\'s Degree':3, 'High School Diploma':0, 'Current K-12':1,'Current Undergraduate':2,'Bachelor\'s Degree':2, 'Associate\'s Degree':3,'Doctoral Degree':4}})
  df1['Income'] = df1.Income.map({'$25,001 - $50,000':1,'over $150,000':5,'$75,000 - $100,000':3,'$50,000 - $74,999':2,'under $25,000':0,'$100,001 - $150,000':4})
  df1['Party'] = df1.Party.map({'Independent':0,'Democrat':1,'Republican':2,'Libertarian':3,'Other':4})
  df1['Gender'] = df1.Gender.map({'Male':0, 'Female':1})

  filter_col = [col for col in df1 if col.startswith('Q')]
  for col in filter_col: 
    # converted yes/no or other feature string values to binary 0 or 1
    #print(col) from the given pdf file possible answers for the questions have been collected
    d = {'No': 0, 'Yes': 1, 'Only-child':0, 'Check!':0,'Nope':1,'Optimist':1,'Pessimist':0,'Mom':0,'Dad':1,'Rent':0,'Own':1, 'Yay people!':0,'Grrr people':1,'Online':0,'In-person':1,'Yes!':1,'Umm...':0,'Socialize':1,'Space':0,'Cautious':1,'Risk-friendly':0,'Mac':1,'PC':0,'Supportive':1,'Demanding':0,'Tunes':0,'Talk':1,'People':0,'Technology':1,'TMI':0,'Mysterious':1,'Start':0,'End':1,'Circumstances':0,'Me':1,'A.M.':0,'P.M.':1,'Happy':0,'Right':1,'Hot headed':0,'Cool headed':1,'Standard hours':0,'Odd hours':1,'Idealist':0,'Pragmatist':1,'Giving':0,'Receiving':1,'Study first':0,'Try first':1,'Science':0,'Art':1,'Public':0,'Private':1}
    df1[col] = df1[col].map(d).fillna(df1[col])
  
  Q1 = df1.quantile(0.25)
  Q3 = df1.quantile(0.75)
  IQR = Q3 - Q1

  df1_TF = (df1 < (Q1 - 1.5 * IQR)) |(df1 > (Q3 + 1.5 * IQR))

  for i in (df1.columns):# to get rid of outliers
    median = df1[i].quantile(0.50) # each feature median is taken
    outlier = df1[i].quantile(0.95) #  95th % value
    outlier_10 = df1[i].quantile(0.10)# 10th % value
    df1[i] = np.where(df1[i] > outlier , outlier, df1[i])# replace value above 95% with median
    df1[i] = np.where(df1[i] < outlier_10, outlier_10,df1[i])# replace value below 10% with median

  #sklearn SimpleImputer is used to fill the missing values. The featuers 'Gender', 
  #'HouseholdStatus', 'Party' are categorical.so instead of mean/median considered 
  #'most_frequent' strategy from SimpleImputer. As YOB' is a numerical feature so it is filled with mean value
  df_mean = df1[features_mean]# features with missing values between 5-20% missing
  df_mean['YOB'].fillna(int(df_mean['YOB'].mean()), inplace=True)
  imp = SimpleImputer(strategy="most_frequent")
  df_mean = imp.fit_transform(df1[features_mean])

  for i in range(1,len(features_mean)):
    x = features_mean[i]
    df1[x] = df_mean[:,i].copy()
  
  df1['YOB'] = df_mean[:,0].copy()
  # taken columns with null values into 'cols_nan'
  cols_nan = df1.columns[df1.isna().any()].tolist()    
  # taken columns with non null values into 'cols_no_nan'
  cols_no_nan = df1.columns.difference(cols_nan).values    
  for col in cols_nan:
    test_data = df1[df1[col].isna()]# rows with null values are taken into testset
    train_data = df1.dropna()#remaining into train
    #Kneighbors classifier is applied
    knr = KNeighborsClassifier(n_neighbors=5).fit(train_data[cols_no_nan], train_data[col])
    #predict each test row with the trained model
    df1.loc[df1[col].isna(), col] = knr.predict(test_data[cols_no_nan])
    #updating columns with nan and without nan for updating train and test with new predicted rows
    cols_nan = df1.columns[df1.isna().any()].tolist()    
    cols_no_nan = df1.columns.difference(cols_nan).values

  df_impu = df1.copy()

  y = df_impu.Happy # separating the class label
  X = df_impu.drop(['Happy'],axis = 1) # taking remaining features

  model = pickle.load(open("/content/drive/My Drive/final_model.pkl",'rb'))
  predictions = model.predict(X)
  score = model.score(X,y)
  print(score)
  
  return(predictions)

In [164]:
import time

start_time = time.clock()
predictions = happiness_prediction(df1)
print (time.clock() - start_time, "seconds")

print(predictions)

0.7018835245724183
10.795162999999974 seconds
[1. 1. 1. ... 0. 0. 1.]


In [165]:
%%time
def error_predict(y,predictions):
  Accuracy = roc_auc_score(df1['Happy'],predictions)
  return (Accuracy)

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 11.7 µs


In [166]:
Accuracy = error_predict(y,predictions)
print(Accuracy)

0.688003663003663


1) Business Problem:  income, education, social support, freedom” are the main criteria to measure happiness . Our dataset is containing all these  features, along with informal questions  which are predicting the happiness of the user and extracting  insights of the data.

One such example is analyzing the factors of a job, which are the largest predictors of workplace satisfaction. Here we can predict overall satisfaction of an employee in the job, based on various attribures like compensation, benifits, flexibility, oppurtunity for advancement etc.

2)ROC AUC answers the question of how well the model discriminates between the two classes. It doesn't tell you anything about the costs of different kinds of errors. 

3) Precision is the percentage of your results which are relevant. recall refers to the percentage of total relevant results correctly classified by your algorithm.

For rare cancer data modeling (whre most of the class labels are nehgative means only few are having cancer), this is an imbalanced data. In this case anything that doesn't account for false-negatives is a crime. In such cases 'Recall' is a better measure than 'precision'.

For YouTube recommendations,happiness prediction false-negatives is less of a concern. In such cases 'Precision' is better here.

so in our case 'Precision' is more important than 'Recall'
